# ATTN: This script should be run AFTER spacy sentence splitting is completed.


# Use spacy to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_SENTENCIZED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

In [3]:
def get_word_num_and_frequency(row, text_col):

    with open(f'{data_dir}punctuations.txt', 'rb') as f:
        custom_punct_chars = pickle.load(f)
    row['Job Description num_words'] = len(str(row[text_col]).split())
    row['Job Description num_unique_words'] = len(set(str(row[text_col]).split()))
    row['Job Description num_chars'] = len(str(row[text_col]))
    row['Job Description num_chars_no_whitespact_and_punt'] = len(
        [
            c
            for c in str(row[text_col])
            if c not in custom_punct_chars and c not in string.punctuation and c != ' '
        ]
    )
    row['Job Description num_punctuations'] = len(
        [
            c
            for c in str(row[text_col])
            if c in custom_punct_chars and c in string.punctuation and c != ' '
        ]
    )

    return row


In [4]:
# Funtion to print df gender and age info
def df_gender_age_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            print(f'{iv} Counts:\n{df[f"{iv}"].value_counts()}')
            print('-'*20)
            print(f'{iv} Percentages:\n{df[f"{iv}"].value_counts(normalize=True).mul(100).round(1).astype(float)}')
            with contextlib.suppress(Exception):
                print('-'*20)
                print(f'{iv} Mean: {df[f"{iv}"].mean().round(2).astype(float)}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {df[f"{iv}"].std().round(2).astype(float)}')
        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [5]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_sentencized.pkl').reset_index(drop=True)


In [6]:
# len=17599
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17599 entries, 0 to 17598
Data columns (total 57 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17599 non-null  object  
 1   Platform                             17599 non-null  object  
 2   Job ID                               17599 non-null  object  
 3   Job Title                            17599 non-null  object  
 4   Company Name                         17597 non-null  object  
 5   Location                             17599 non-null  object  
 6   Job Description                      17599 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17017 non-null  object  
 9   Company URL                          15959 non-null  object  
 10  Job URL                              17599 non-null  object  
 11  Job Age        

In [7]:
# Job Ad info
df_gender_age_info(df_jobs)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17599 entries, 0 to 17598
Data columns (total 57 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17599 non-null  object  
 1   Platform                             17599 non-null  object  
 2   Job ID                               17599 non-null  object  
 3   Job Title                            17599 non-null  object  
 4   Company Name                         17597 non-null  object  
 5   Location                             17599 non-null  object  
 6   Job Description                      17599 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17017 non-null  object  
 9   Company URL                          15959 non-null  object  
 10  Job URL                              17599 non-null  object  
 11  Job 

In [8]:
# Explode df so that every row is one sentence
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True).reset_index(drop=True)


In [9]:
# 194820
len(df_jobs)


194820

In [10]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 57 columns):
 #   Column                               Non-Null Count   Dtype   
---  ------                               --------------   -----   
 0   Search Keyword                       194820 non-null  object  
 1   Platform                             194820 non-null  object  
 2   Job ID                               194820 non-null  object  
 3   Job Title                            194820 non-null  object  
 4   Company Name                         194816 non-null  object  
 5   Location                             194820 non-null  object  
 6   Job Description                      194820 non-null  object  
 7   Rating                               170120 non-null  float64 
 8   Employment Type                      163313 non-null  object  
 9   Company URL                          192654 non-null  object  
 10  Job URL                              194820 non-null  object  
 11  

In [11]:
df_jobs['Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: job_sentence.strip().lower()
)


progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

In [12]:
df_jobs[['Job Description spacy_sentencized', 'Job Description spacy_sentencized_lower']].head()


,Job Description spacy_sentencized,Job Description spacy_sentencized_lower
0,About Our Client,about our client
1,The Global KYC organisation is part of ING's C...,the global kyc organisation is part of ing's c...
2,Its purpose is Enabling people and organisatio...,its purpose is enabling people and organisatio...
3,Our Global KYC organisation is a first line of...,our global kyc organisation is a first line of...
4,In our Global KYC organisation you will be wor...,in our global kyc organisation you will be wor...


In [13]:
%%time
# Spacy tokenize
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.text.strip().lower())
        for token in nlp.tokenizer(job_sentence)
        if len(token) != 0
        and not token.is_space
        and not token.is_stop
        and not token.is_punct
        and not token.is_bracket
        and not token.like_email
        and not token.text in custom_punct_chars
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

CPU times: user 38.6 s, sys: 3.55 s, total: 42.2 s
Wall time: 56.3 s


In [14]:
df_jobs['Job Description spacy_sentencized_cleaned'] = df_jobs['Job Description spacy_tokenized'].str.join(' ')


In [15]:
%%time
# Get sentence word frequencies
df_jobs = df_jobs.progress_apply(
    lambda row: get_word_num_and_frequency(
        row=row, text_col='Job Description spacy_sentencized'
    ), 
    axis='columns',
)


progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

CPU times: user 3min 13s, sys: 12.4 s, total: 3min 25s
Wall time: 3min 58s


In [16]:
df_jobs[
    [
        'Job Description spacy_sentencized',
        'Job Description num_words', 'Job Description num_unique_words',
        'Job Description num_chars', 'Job Description num_chars_no_whitespact_and_punt'
    ]
].head()

,Job Description spacy_sentencized,Job Description num_words,Job Description num_unique_words,Job Description num_chars,Job Description num_chars_no_whitespact_and_punt
0,About Our Client,3,3,16,14
1,The Global KYC organisation is part of ING's C...,10,10,56,45
2,Its purpose is Enabling people and organisatio...,20,19,131,111
3,Our Global KYC organisation is a first line of...,34,31,239,203
4,In our Global KYC organisation you will be wor...,18,18,128,109


In [17]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership', 'Language', 'Dutch Requirement', 'English Requirement', 'Sector Code', 'Sector', 'Keywords Count', '% per Sector', '% per Social Category', '% per Workforce', 'Female Count (x 1000)', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Male Count (x 1000)', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector Count (x 1000)', '% Sector per Workforce', 'Gender_Female', 'Gender_Ma

In [18]:
# Job Sentence info
df_gender_age_info(df_jobs)



DF INFO:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 65 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object

In [19]:
with open(f'{data_dir}df_jobs_len.txt', 'wb') as f:
    pickle.dump(len(df_jobs), f)

In [20]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


# Use NLTK to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [21]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [22]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [23]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl').reset_index(drop=True)


In [24]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 65 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object 
 9   Comp

In [25]:
%%time
# Tokenize with NLTK
# stop_words = set(stopwords.words('english'))
# punctuations = list(string.punctuation)
# lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()

df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

CPU times: user 6min, sys: 1min 59s, total: 8min
Wall time: 8min 19s


In [26]:
df_jobs['Job Description nltk_tokenized'].head()


0                                             [client]
1    [global, kyc, organisation, part, ing, 's, coo...
2    [purpose, enabling, people, organisations, use...
3    [global, kyc, organisation, first, line, defen...
4    [global, kyc, organisation, working, many, col...
Name: Job Description nltk_tokenized, dtype: object

In [27]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 66 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object 
 9   Comp

In [28]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


# Use gensim to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [29]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [30]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [31]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl').reset_index(drop=True)


In [32]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 66 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object 
 9   Comp

In [33]:
%%time
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

CPU times: user 46.5 s, sys: 1.78 s, total: 48.3 s
Wall time: 49.4 s


In [34]:
df_jobs['Job Description gensim_tokenized'].head()


0                                             [client]
1             [global, kyc, organis, ing, coo, domain]
2    [purpos, enabl, peopl, organis, us, bank, serv...
3    [global, kyc, organis, line, defenc, depart, p...
4    [global, kyc, organis, work, colleagu, differ,...
Name: Job Description gensim_tokenized, dtype: object

In [35]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 67 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object 
 9   Comp

In [36]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


# Use BERT to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [37]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [38]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [39]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl').reset_index(drop=True)


In [40]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194820 entries, 0 to 194819
Data columns (total 67 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   Search Keyword                                    194820 non-null  object 
 1   Platform                                          194820 non-null  object 
 2   Job ID                                            194820 non-null  object 
 3   Job Title                                         194820 non-null  object 
 4   Company Name                                      194816 non-null  object 
 5   Location                                          194820 non-null  object 
 6   Job Description                                   194820 non-null  object 
 7   Rating                                            170120 non-null  float64
 8   Employment Type                                   163313 non-null  object 
 9   Comp

In [41]:
%%time
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name).to(device)

df_jobs['Job Description bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: bert_tokenizer.tokenize(str(sentence))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.csv', index=False)


Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

progress-bar:   0%|          | 0/194820 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1539 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 50.2 s, sys: 2.49 s, total: 52.7 s
Wall time: 55 s


In [42]:
df_jobs['Job Description bert_tokenized'].head()


0                                 [about, our, client]
1    [the, global, ky, ##c, organisation, is, part,...
2    [its, purpose, is, enabling, people, and, orga...
3    [our, global, ky, ##c, organisation, is, a, fi...
4    [in, our, global, ky, ##c, organisation, you, ...
Name: Job Description bert_tokenized, dtype: object

In [43]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.csv', index=False)
